<a href="https://colab.research.google.com/github/abhi1021/resnet50-imagenet-1k/blob/main/train.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Training ImageNette Model (Google Colab)
This notebook trains the ResNet-50 model on ImageNette-160 (10 classes, epochs=3)

## Download ImageNette Dataset

In [1]:
# Clone the repository
!git clone https://github.com/abhi1021/resnet50-imagenet-1k.git

Cloning into 'resnet50-imagenet-1k'...
remote: Enumerating objects: 85, done.
remote: Counting objects: 100% (85/85), done.
remote: Compressing objects: 100% (59/59), done.
remote: Total 85 (delta 37), reused 73 (delta 25), pack-reused 0 (from 0)
Receiving objects: 100% (85/85), 367.52 KiB | 8.96 MiB/s, done.
Resolving deltas: 100% (37/37), done.


In [2]:
# Navigate to the repository directory
%cd resnet50-imagenet-1k

/content/resnet50-imagenet-1k


In [ ]:
# Download and extract ImageNette dataset
!wget https://s3.amazonaws.com/fast-ai-imageclas/imagenette2-160.tgz
!tar -xzf imagenette2-160.tgz
!ls -la imagenette2-160

## Verify Dataset Structure

In [ ]:
# Check dataset structure (should have train and val folders)
!ls -la imagenette2-160/train | head -15
!echo "\n"
!ls -la imagenette2-160/val | head -15

## Import Libraries and Check GPU

In [ ]:
import torch
import sys
from train import ImageClassificationTrainer

# Check if CUDA is available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")
if device.type == 'cuda':
    print(f"GPU: {torch.cuda.get_device_name(0)}")
    print(f"Memory Allocated: {torch.cuda.memory_allocated(0) / 1024**2:.2f} MB")
    print(f"Memory Cached: {torch.cuda.memory_reserved(0) / 1024**2:.2f} MB")
else:
    print("WARNING: Running on CPU. Training will be slower.")
    print("In Colab, go to Runtime > Change runtime type > Hardware accelerator > GPU")

## Initialize and Train ImageNette Model with ResNet-50

In [ ]:
# Create trainer with specified settings
# Configuration:
#   - Model: ResNet-50 (25.6M parameters)
#   - Dataset: ImageNette (10 classes, 160x160 images)
#   - Epochs: 3
#   - Batch Size: 128 (adjusted for Colab GPU)
#   - Learning Rate: Warmup (0.01→0.1) + Cosine annealing
#   - MixUp augmentation: Enabled (alpha=0.2)
#   - Label smoothing: 0.1
#   - Mixed precision: Enabled
#   - Gradient clipping: 1.0
trainer = ImageClassificationTrainer(
    model_name='resnet50',
    dataset='imagenet',
    data_dir='imagenette2-160',
    epochs=3,
    batch_size=128,
    num_workers=2
)

print("Starting ImageNette Model Training with ResNet-50")
print("="*70)
print(f"Configuration:")
print(f"  - Model: ResNet-50 (25.6M parameters)")
print(f"  - Dataset: ImageNette (10 classes)")
print(f"  - Image Size: 160x160 (resized to 224x224)")
print(f"  - Batch Size: 128")
print(f"  - Epochs: 3")
print(f"  - Optimizer: SGD (momentum=0.9, weight_decay=1e-3)")
print(f"  - Learning Rate: Warmup (0.01→0.1) + Cosine annealing")
print(f"  - MixUp: Enabled (alpha=0.2)")
print(f"  - Label Smoothing: 0.1")
print(f"  - Mixed Precision: Enabled")
print(f"  - Gradient Clipping: 1.0")
print("="*70)

In [7]:
# Run training
final_accuracy = trainer.run()


Training wideresnet for 3 epochs on CIFAR-100

Model Architecture Summary
Device: cuda
Model: wideresnet
Mixed Precision: True
MixUp: True (alpha=0.2)
Label Smoothing: 0.1

Model Summary:



Epoch 1 Loss=4.4725 Acc=3.84% LR=0.027908: 100%|██████████| 196/196 [01:47<00:00,  1.83it/s]



Test set: Average loss: 4.1322, Accuracy: 698/10000 (6.98%)

*** New best model! Test Accuracy: 6.98% ***
✓ Checkpoint saved: best_model.pth
Best Test Accuracy so far: 6.98%



Epoch 2 Loss=3.7481 Acc=9.68% LR=0.045908: 100%|██████████| 196/196 [01:47<00:00,  1.82it/s]



Test set: Average loss: 3.9508, Accuracy: 1061/10000 (10.61%)

*** New best model! Test Accuracy: 10.61% ***
✓ Checkpoint saved: best_model.pth
Best Test Accuracy so far: 10.61%



Epoch 3 Loss=3.4351 Acc=15.67% LR=0.063908: 100%|██████████| 196/196 [01:47<00:00,  1.82it/s]



Test set: Average loss: 3.3465, Accuracy: 1930/10000 (19.30%)

*** New best model! Test Accuracy: 19.30% ***
✓ Checkpoint saved: best_model.pth
Best Test Accuracy so far: 19.30%


📦 Saving final model...
✓ Checkpoint saved: final_model.pth

Training completed. Best test accuracy: 19.30%

TRAINING AND TESTING LOSS
   (Y)     ^
4.87331457 |
4.62964884 | ⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀
4.38598311 | ⠤⠤⠤⣀⣀⣀⣀⣀⣀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀
4.14231739 | ⣀⣀⣀⣀⣀⣀⠀⠀⠀⠉⠉⠉⠉⠉⠉⠒⠒⠒⠒⠒⠢⠤⠤⠤⠤⠤⢄⣀⣀⣀⣀⣀⡀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀
3.89865166 | ⠀⠀⠀⠀⠀⠀⠉⠉⠉⠉⠉⠉⠉⠉⠉⠉⠉⠑⠒⠒⠒⠒⠒⠒⠒⠒⠒⠒⠒⠢⠤⠤⠬⠭⠭⠭⠭⣉⣉⣑⡒⠒⠒⠒⠒⠒⠒⠒⠢⠤⠤⠤⠤⠤⠤⠤⠤⣀⣀⣀⣀⣀⣀⣀⣀⣀⠀⠀⠀⠀
3.65498593 | ⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠈⠉⠉⠉⠒⠒⠒⠢⠤⠤⠤⣀⣀⣀⡀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠉⠉⠉⠉
3.41132020 | ⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠈⠉⠉⠉⠒⠒⠒⠢⠤⠤⠤⣀⣀⣀⡀⠀
3.16765447 | ⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠈⠉
2.92398874 | ⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀

## Training Summary

In [ ]:
print("\n" + "="*70)
print("TRAINING COMPLETED")
print("="*70)
print(f"Model: ResNet-50")
print(f"Dataset: ImageNette (10 classes)")
print(f"Final Best Test Accuracy: {final_accuracy:.2f}%")
print("="*70)
print("\nCheckpoints saved in ./checkpoints/")
print("  - best_model.pth (highest test accuracy)")
print("  - final_model.pth (final epoch)")
print("  - training_curves.png (visualization)")
print("  - metrics.json (complete training history)")